In [ ]:
import ipaddress
import socket

# Get the IP address of the user's device
def get_device_ip():
    hostname = socket.gethostname()
    ip_address = socket.gethostbyname(hostname)
    print("Your Computer Name is: " + hostname)
    print("Your Computer IP Address is: " + ip_address)
    return ip_address

# Get the IP address of a given website
def get_ip_address(website_url):
    try:
        ip_address = socket.gethostbyname(website_url)
        print(f"The IP address of {website_url} is {ip_address}")
        return ip_address
    except socket.gaierror:
        print(f"Unable to get the IP address for {website_url}")
        return None

# Analyze IP details including network, netmask, and usable hosts
def analyse_ip(ip_str, new_prefix=None):
    # Create an IP interface object
    ip = ipaddress.ip_interface(ip_str)
    network = ip.network
    
    print(f"Address: {ip.ip}")
    print(f"Network: {network}")
    print(f"Netmask: {network.netmask}")
    print(f"Broadcast Address: {network.broadcast_address}")
    print(f"First Usable Host: {list(network.hosts())[0] if network.num_addresses > 2 else 'N/A'}")
    print(f"Last Usable Host: {list(network.hosts())[-1] if network.num_addresses > 2 else 'N/A'}")
    print(f"Number of Usable Hosts: {network.num_addresses - 2 if network.num_addresses > 2 else 0}")
    print(f"Is Private: {ip.ip.is_private}")
    print(f"Is Global: {ip.ip.is_global}")
    
    # List all hosts in the network (only for small networks)
    if network.num_addresses < 16:
        print("\nHosts in network:")
        for host in network.hosts():
            print(host)
    
    # Compare with a different CIDR prefix if provided
    if new_prefix:
        new_network = ipaddress.ip_network(f"{ip.ip}/{new_prefix}", strict=False)
        print(f"\nChanged IP to CIDR /{new_prefix}:")
        print(f"New Network: {new_network}")
        print(f"New Netmask: {new_network.netmask}")
        print(f"New Broadcast Address: {new_network.broadcast_address}")
        print(f"New Usable Hosts: {new_network.num_addresses - 2 if new_network.num_addresses > 2 else 0}")

# Create a subnet plan for different departments
def create_subnets(base_network, departments):
    subnets = {}
    sorted_departments = sorted(departments.items(), key=lambda x: x[1], reverse=True) # Sort by largest host needs
    network = ipaddress.ip_network(base_network)
    
    for dept, host_count in sorted_departments:
        required_hosts = host_count + 2  # Including network and broadcast addresses
        new_prefix = 32 - (required_hosts - 1).bit_length()
        subnet = next(network.subnets(new_prefix=new_prefix))
        subnets[dept] = subnet
        network = ipaddress.ip_network(f"{subnet.broadcast_address + 1}/{network.prefixlen}", strict=False)
    
    # Print subnet details for each department
    for dept, subnet in subnets.items():
        print(f"{dept} Subnet: {subnet}")
        print(f"Netmask: {subnet.netmask}")
        print(f"First Usable Host: {list(subnet.hosts())[0]}")
        print(f"Last Usable Host: {list(subnet.hosts())[-1]}")
        print(f"Number of Usable Hosts: {subnet.num_addresses - 2}\n")

# Get and analyze the device's IP
device_ip = get_device_ip()
analyse_ip(f'{device_ip}/24', new_prefix=28)

# Get and analyze university website IP
university_url = "www.gold.ac.uk"
university_ip = get_ip_address(university_url)
if university_ip:
    analyse_ip(f'{university_ip}/24', new_prefix=28)

# Create subnetting plan for the company departments
company_network = "172.16.0.0/16"
departments = {
    "Engineering": 30,
    "Marketing": 15,
    "Finance": 10,
    "HR": 5
}
create_subnets(company_network, departments)


Your Computer Name is: LAPTOP-N91VRVMP
Your Computer IP Address is: 172.20.10.10
Address: 172.20.10.10
Network: 172.20.10.0/24
Netmask: 255.255.255.0
Broadcast Address: 172.20.10.255
First Usable Host: 172.20.10.1
Last Usable Host: 172.20.10.254
Number of Usable Hosts: 254
Is Private: True
Is Global: False

Changed IP to CIDR /28:
New Network: 172.20.10.0/28
New Netmask: 255.255.255.240
New Broadcast Address: 172.20.10.15
New Usable Hosts: 14
The IP address of www.gold.ac.uk is 159.100.136.66
Address: 159.100.136.66
Network: 159.100.136.0/24
Netmask: 255.255.255.0
Broadcast Address: 159.100.136.255
First Usable Host: 159.100.136.1
Last Usable Host: 159.100.136.254
Number of Usable Hosts: 254
Is Private: False
Is Global: True

Changed IP to CIDR /28:
New Network: 159.100.136.64/28
New Netmask: 255.255.255.240
New Broadcast Address: 159.100.136.79
New Usable Hosts: 14
Engineering Subnet: 172.16.0.0/27
Netmask: 255.255.255.224
First Usable Host: 172.16.0.1
Last Usable Host: 172.16.0.30
Nu